In [57]:
import numpy as np
from keras.models import Sequential
from keras.models import model_from_yaml
from keras.models import load_model
from keras.layers import LSTM, Dense, Activation, GRU
from keras.callbacks import ModelCheckpoint
from random import randint

In [58]:
with open("lyrics.csv") as corpus_file:
    corpus = corpus_file.read()
    corpus = corpus.lower()

# Get a unique identifier for each char in the corpus, then make some dicts to ease encoding and decoding
chars = sorted(list(set(corpus)))
num_chars = len(chars)
encoding = {c: i for i, c in enumerate(chars)}
decoding = {i: c for i, c in enumerate(chars)}

In [59]:

with open('model.yaml', 'r') as model_file:
    # read yaml file 
    model = model_from_yaml(model_file)

model.load_weights('weights-20.hdf5')
model.compile(loss='categorical_crossentropy', optimizer='adam') 
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               86016     
_________________________________________________________________
dense_2 (Dense)              (None, 39)                5031      
_________________________________________________________________
activation_2 (Activation)    (None, 39)                0         
Total params: 91,047
Trainable params: 91,047
Non-trainable params: 0
_________________________________________________________________


In [60]:
sentence_length = 20
def generate_text(seed_pattern):
    X = np.zeros((1, sentence_length, num_chars), dtype=np.bool)
    for i, char in enumerate(seed_pattern):
        X[0, i, encoding[char]] = 1
        
    generated_text = ''
    
    for i in range(500):
        prediction = np.argmax(model.predict(X, verbose=0))
        generated_text += decoding[prediction]
        
        activations = np.zeros((1, 1, num_chars), dtype=np.bool)
        activations[0,0, prediction] = 1
        X = np.concatenate((X[:,1:,:], activations), axis=1)
    return generated_text

def make_seed(seed=''):
    if seed:
        seed_length = len(seed)
        pattern = ''
        for i in range(0, sentence_length):
            patten += seed[i % seed_length]
    else:
        seed = randint(0, len(corpus) - sentence_length)
        pattern = corpus[seed:seed + sentence_length]
        
    return pattern
        

In [61]:
seed = make_seed()
text = generate_text(seed)
print(text)

ValueError: Error when checking input: expected lstm_2_input to have shape (20, 39) but got array with shape (20, 352)